In [ ]:
# tweet_pipeline.py
# Run with: python tweet_pipeline.py
import re
import json
from pprint import pprint
import nltk
import string 

from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.chunk import RegexpParser

In [3]:
# ensure minimal NLTK data (will download if missing)
for pkg in ("punkt", "averaged_perceptron_tagger", "wordnet", "omw-1.4", "stopwords"):
    try:
        nltk.data.find(pkg)
    except LookupError:
        nltk.download(pkg.split("/")[-1] if "/" in pkg else pkg)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aditikulkarni/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aditikulkarni/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aditikulkarni/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aditikulkarni/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aditikulkarni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Data
tweets = [
    "Just finished reading an amazing book on AI 🤖📚 #MachineLearning #AI.",
    "What a beautiful morning! ☀️ Feeling super motivated to start the day 💪 #GoodVibes",
    "Ugh...@user my laptop crashed again right before my deadline 😤 #MondayBlues",
    "brb grabbing some ☕️ lol can’t start work without caffeine 😂"
]

In [5]:
# small contraction map sufficient for demo
contraction_map = {
    "can't": "can not", "cant": "can not",
    "won't": "will not",
    "didn't": "did not", "didnt": "did not",
    "don't": "do not", "dont": "do not",
    "can't": "can not", "can't": "can not",
    "can't": "can not",
    "can't": "can not",
    "can't": "can not",
    "can’t": "can not", "didn’t": "did not", "don’t": "do not",
    "brb": "be right back", "lol": "laughing out loud", "u": "you"
}

# emoji map (small)
emoji_map = {
    "🤖": "robot", "📚": "books", "☀️": "sun", "💪": "strong",
    "😤": "annoyed", "☕️": "coffee", "😂": "laugh", "😊": "smile"
}

# chunk grammar (simple)
GRAMMAR = r"""
  NP: {<DT|PRP\$>?<JJ.*>*<NN.*>+}   # determiner/adjectives + nouns
  PP: {<IN><NP>}                    # preposition + NP
  VP: {<VB.*><NP|PP|CLAUSE>+$}      # verb phrase (simple)
"""

In [ ]:
# tokenizer
tokenizer = TweetTokenizer()

# stemmers / lemmatizer
porter = PorterStemmer()
snow = SnowballStemmer("english")
lanc = LancasterStemmer()
wnl = WordNetLemmatizer()

# chunker
chunker = RegexpParser(GRAMMAR)
chunk = 

# helper functions
def replace_emojis(text):
    for emoji, word in emoji_map.items():
        text = text.replace(emoji, " " + word + " ")
    return text

def expand_contractions(text):
    for short, long in contraction_map.items():
        text = re.sub(r"\b" + re.escape(short) + r"\b", long, text)
    return text

def remove_punctuation(text):
    # keep hashtags and mentions
    for c in text:
        if c in string.punctuation and c not in ("#","@"):
            text = text.replace(c, " ")
    return text

def normalize_text(text):
    # lower case, emoji -> word, expand contractions, separate punctuation (keep hashtags/mentions)
    text = text.lower()
    text = remove_punctuation(text)

    #text = re.sub(r"\.{2,}", ".", text)            # collapse dots
    # space around punctuation for clearer tokenization
    #text = re.sub(r"([!?.,])", r" \1 ", text)
    #text = re.sub(r"\s+", " ", text).strip()

    text = expand_contractions(text)
    text = replace_emojis(text)

    return text

def remove_stopwords(words):
    from nltk.corpus import stopwords
    stopset = set(stopwords.words("english"))
    filtered = [w for w in words if w.lower() not in stopset]
    return filtered

def tokenize(text):
    return tokenizer.tokenize(text)

# convert NLTK treebank tag to WordNet POS (for lemmatizer)
from nltk.corpus import wordnet as wn
def treebank_to_wordnet_pos(tb_tag):
    if tb_tag.startswith("J"):
        return wn.ADJ
    if tb_tag.startswith("V"):
        return wn.VERB
    if tb_tag.startswith("N"):
        return wn.NOUN
    if tb_tag.startswith("R"):
        return wn.ADV
    return wn.NOUN

def stem_words(words):
    return {
        "porter": [porter.stem(w) for w in words],
        "snowball": [snow.stem(w) for w in words],
        "lancaster": [lanc.stem(w) for w in words]
    }

def lemmatize_words(words, pos_tags):
    # pos_tags: list of treebank tags from pos_tag
    lemmas = []
    for w, t in zip(words, pos_tags):
        wnpos = treebank_to_wordnet_pos(t)
        lemmas.append(wnl.lemmatize(w, wnpos))
    return lemmas

def chunk_pos_tags(pos_tags):
    tree = chunker.parse(pos_tags)
    return tree

# very simple heuristic dependency extractor (not a real parser)
def simple_dependencies(pos_tags):
    words = [w for w,t in pos_tags]
    tags = [t for w,t in pos_tags]
    deps = []
    # choose first verb as head (simple)
    v_idx = next((i for i,t in enumerate(tags) if t.startswith("V")), None)
    if v_idx is None:
        return deps
    head = words[v_idx]
    # subject: nearest noun/pronoun before verb
    subj_idx = next((i for i in range(v_idx-1, -1, -1) if tags[i].startswith("N") or tags[i] in ("PRP","NNP")), None)
    if subj_idx is not None:
        deps.append((head, words[subj_idx], "nsubj"))
    # direct object: first noun after verb
    dobj_idx = next((i for i in range(v_idx+1, len(words)) if tags[i].startswith("N")), None)
    if dobj_idx is not None:
        deps.append((head, words[dobj_idx], "dobj"))
    # negation: 'not' or 'no' attaches to head
    for i,w in enumerate(words):
        if w.lower() in ("not","no","n't"):
            deps.append((head, w, "neg"))
    # adjectival modifiers attach to nearest noun
    for i,(w,t) in enumerate(pos_tags):
        if t.startswith("JJ"):
            # find nearest noun to right then left
            target = None
            for j in range(i+1, len(words)):
                if tags[j].startswith("N"):
                    target = words[j]; break
            if not target:
                for j in range(i-1, -1, -1):
                    if tags[j].startswith("N"):
                        target = words[j]; break
            if target:
                deps.append((target, w, "amod"))
    # time words
    for tword in ("yesterday","today","tomorrow","now"):
        if tword in [w.lower() for w in words]:
            deps.append((head, tword, "time"))
    return deps

def agreement_check(pos_tags):
    words = [w for w,t in pos_tags]; tags=[t for w,t in pos_tags]
    v_idx = next((i for i,t in enumerate(tags) if t.startswith("V")), None)
    if v_idx is None:
        return ["No verb found"]
    subj_idx = next((i for i in range(v_idx-1, -1, -1) if tags[i].startswith("N") or tags[i] in ("PRP","NNP")), None)
    if subj_idx is None:
        return ["No subject found"]
    subj = words[subj_idx]; subj_tag = tags[subj_idx]
    verb = words[v_idx]; verb_tag = tags[v_idx]
    # crude plural detection
    subj_plural = subj_tag in ("NNS","NNPS") or subj.lower().endswith("s")
    verb_3sg = verb_tag == "VBZ" or (verb.endswith("s") and not verb.endswith("ss"))
    if subj_plural and verb_3sg:
        return [f"Mismatch: plural subject '{subj}' with 3rd-person-singular verb '{verb}'"]
    if (not subj_plural) and verb_3sg and subj.lower() in ("i","you"):
        return [f"Possible mismatch with subject '{subj}' and verb '{verb}'"]
    return ["Agreement heuristic OK"]

def simple_srl(deps, pos_tags):
    roles = {}
    for h,d,r in deps:
        if r=="nsubj":
            roles["agent"]=d
        elif r=="dobj":
            # theme (object)
            roles["theme"]=d
        elif r=="iobj":
            roles["recipient"]=d
        elif r=="neg":
            roles["polarity"]="negative"
        elif r=="amod":
            roles.setdefault("adjectives", []).append(d)
        elif r=="time":
            roles["time"]=d
    return roles




In [10]:
# pipeline
from nltk.corpus import stopwords
stopset = set(stopwords.words("english"))

results = []
for tweet in tweets:
    entry = {"raw": tweet}
    # 1. normalization
    norm = normalize_text(tweet)
    entry["normalized"] = norm
    # 2. tokenization
    toks = tokenize(norm)
    entry["tokens"] = toks
    # 3. stopword filtering (but keep negations)
    filtered = remove_stopwords(toks)
    entry["filtered_tokens"] = filtered
    # 4. POS tagging
    pos = pos_tag(filtered)
    entry["pos_tags"] = pos
    # 5. stemming
    stems = stem_words([w for w,t in pos])
    entry["stems"] = stems
    # 6. lemmatization (wordnet)
    lemmas = lemmatize_words([w for w,t in pos], [t for w,t in pos])
    entry["lemmas"] = lemmas
    # 7. chunking
    chunk_tree = chunk_pos_tags(pos)
    entry["chunk_tree_str"] = str(chunk_tree)
    # 8. simple dependencies
    deps = simple_dependencies(pos)
    entry["dependencies"] = deps
    # 9. agreement check
    entry["agreement"] = agreement_check(pos)
    # 10. simple SRL
    entry["srl"] = simple_srl(deps, pos)
    results.append(entry)

# Pretty print each stage per tweet:
for i, r in enumerate(results, 1):
    print("="*80)
    print(f"TWEET #{i}: {r['raw']}\n")
    print("1) Normalized ->\n", r["normalized"])
    print("\n2) Tokenized ->\n", r["tokens"])
    print("\n3) Stopword filtered (keeps negations) ->\n", r["filtered_tokens"])
    print("\n4) POS tags ->\n", r["pos_tags"])
    print("\n5) Stems ->\n")
    print(" Porter:", r["stems"]["porter"])
    print(" Snowball:", r["stems"]["snowball"])
    print(" Lancaster:", r["stems"]["lancaster"])
    print("\n6) Lemmas ->\n", r["lemmas"])
    print("\n7) Chunk (tree) ->\n", r["chunk_tree_str"])
    print("\n8) Heuristic dependencies ->\n", r["dependencies"])
    print("\n9) Agreement check ->\n", r["agreement"])
    print("\n10) Simple SRL ->\n", r["srl"])

# save results for later inspection
with open("tweet_pipeline_results.json", "w", encoding="utf8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("\nSaved pipeline results -> tweet_pipeline_results.json")

TWEET #1: Just finished reading an amazing book on AI 🤖📚 #MachineLearning #AI.

1) Normalized ->
 just finished reading an amazing book on ai  robot  books  #machinelearning #ai 

2) Tokenized ->
 ['just', 'finished', 'reading', 'an', 'amazing', 'book', 'on', 'ai', 'robot', 'books', '#machinelearning', '#ai']

3) Stopword filtered (keeps negations) ->
 ['finished', 'reading', 'amazing', 'book', 'ai', 'robot', 'books', '#machinelearning', '#ai']

4) POS tags ->
 [('finished', 'VBN'), ('reading', 'NN'), ('amazing', 'VBG'), ('book', 'NN'), ('ai', 'VBP'), ('robot', 'NN'), ('books', 'NNS'), ('#machinelearning', 'VBG'), ('#ai', 'NN')]

5) Stems ->

 Porter: ['finish', 'read', 'amaz', 'book', 'ai', 'robot', 'book', '#machinelearn', '#ai']
 Snowball: ['finish', 'read', 'amaz', 'book', 'ai', 'robot', 'book', '#machinelearn', '#ai']
 Lancaster: ['fin', 'read', 'amaz', 'book', 'ai', 'robot', 'book', '#machinelearning', '#ai']

6) Lemmas ->
 ['finish', 'reading', 'amaze', 'book', 'ai', 'robot', 'b